In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.path import Path

from joblib import Parallel, delayed

In [2]:
from enum import Enum

class PolygonType(Enum):
    CLOSED = 'closed'
    OPEN = 'open'
    TEXTURE = 'textured'
    TEXTURE_WITH_CONTOUR = 'texture with contour'
    DIRECTION = 'directionality'

In [3]:
stack = 'MD594'

first_bs_sec, last_bs_sec = section_range_lookup[stack]

In [4]:
autoAnnotation_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_autoAnnotationViz'

autoAnnotation_dir = os.path.join(autoAnnotation_rootdir, stack)
if not os.path.exists(autoAnnotation_dir):
    os.makedirs(autoAnnotation_dir)

In [40]:
section_contains_label = {}
for sec in range(first_bs_sec, last_bs_sec+1):
    dm = DataManager(stack=stack, section=sec)
    try:
        user, ts, _, res = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
        section_contains_label[sec] = set([lm['label'] for lm in res])
    except:
        pass

username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username yuncong does not have any labelings
username y

In [6]:
scoremap = None

In [7]:
def score_transform_affine(tx, ty, theta, scx, scy, a, b, init_cnt):
    
    xmin, ymin = np.min(init_cnt, axis=0)
    xmax, ymax = np.max(init_cnt, axis=0)
    cx, cy = np.mean(init_cnt, axis=0)
    center = np.array([cx, cy])

    xs, ys = np.meshgrid(np.arange(xmin, xmax+1), np.arange(ymin, ymax+1))
    pts = np.c_[xs.flat, ys.flat]
    nzs = pts[np.where(Path(init_cnt).contains_points(pts))[0]]
        
    cos_z = np.cos(theta)
    sin_z = np.sin(theta)
    R = np.array([[scx*cos_z, -scx*sin_z], [scy*sin_z, scy*cos_z]])
    
    test_xs, test_ys = (np.dot(np.dot([[1, a], [b, 1]], R), nzs.T - np.array([cx,cy])[:,np.newaxis]) + \
                        np.array([tx+cx,ty+cy])[:,np.newaxis]).astype(np.int)

    valid = (test_xs >= 0) & (test_ys >= 0) & \
            (test_xs < scoremap.shape[1]) & (test_ys < scoremap.shape[0])
    
    score = scoremap[test_ys[valid], test_xs[valid]].sum() * scx * scy

    return score

In [13]:
def grid_search_affine(init_cnt, init_params=(0, 0, 0, 1., 1., 0, 0), iter_num=10, 
                       sigma_tx0=300, sigma_ty0=300, sigma_theta_z0=0, 
                                        sigma_scx0=0, sigma_scy0=0,
                                        sigma_a0=0, sigma_b0=0):
    
    params_best_upToNow = init_params
    score_best_upToNow = 0

    for iteration in range(iter_num):

#         print iteration

        init_tx, init_ty, init_theta_z, init_scalex, init_scaley, init_a, init_b = params_best_upToNow

        n = int(1000*np.exp(-iteration/10.))
        if n < 10:
            break

        sigma_tx = sigma_tx0*np.exp(-iteration/3.)
        sigma_ty = sigma_ty0*np.exp(-iteration/3.)
        sigma_theta_z = np.deg2rad(sigma_theta_z0*np.exp(-iteration/.6))
        sigma_scx = sigma_scx0*np.exp(-iteration/20.)
        sigma_scy = sigma_scy0*np.exp(-iteration/20.)
        sigma_a = sigma_a0*np.exp(-iteration/5.)
        sigma_b = sigma_b0*np.exp(-iteration/5.)

#         print sigma_tx, sigma_ty, sigma_theta_z, sigma_scx, sigma_scy
            
        tx_grid = init_tx + 2*sigma_tx*np.random.random(n) - sigma_tx
        ty_grid = init_ty + 2*sigma_ty*np.random.random(n) - sigma_ty
        theta_z_grid = init_theta_z + 2*sigma_theta_z*np.random.random(n) - sigma_theta_z
        scalex_grid = init_scalex * np.exp(2*sigma_scx*np.random.random(n) - sigma_scx)
        scaley_grid = init_scaley * np.exp(2*sigma_scy*np.random.random(n) - sigma_scy)
        a_grid = init_a + 2*sigma_a*np.random.random(n) - sigma_a
        b_grid = init_b + 2*sigma_b*np.random.random(n) - sigma_b
        

        scores = Parallel(n_jobs=16)(delayed(score_transform_affine)(tx, ty, theta_z, scx, scy, a, b, init_cnt) 
                                     for tx, ty, theta_z, scx, scy, a, b in zip(tx_grid, ty_grid, theta_z_grid,
                                                               scalex_grid, scaley_grid, a_grid, b_grid))

#         scores = [score_transform_affine(tx, ty, theta_z, scx, scy, a, b, init_cnt) 
#                                      for tx, ty, theta_z, scx, scy, a, b in zip(tx_grid, ty_grid, theta_z_grid,
#                                                                scalex_grid, scaley_grid, a_grid, b_grid)]
        
        
        score_best = np.max(scores)

        tx_best, ty_best, theta_z_best, scalex_best, scaley_best, a_best, b_best = \
            zip(tx_grid, ty_grid, theta_z_grid, scalex_grid, scaley_grid, a_grid, b_grid)[np.argmax(scores)]
            
        
        scalex_best = min(2., max(scalex_best, 0.1))
        scaley_best = min(2., max(scaley_best, 0.1))
            
            
        if score_best > score_best_upToNow:
#             print score_best_upToNow, score_best

            score_best_upToNow = score_best
            params_best_upToNow = tx_best, ty_best, theta_z_best, scalex_best, scaley_best, a_best, b_best

#             print tx_best, ty_best, np.rad2deg(theta_z_best), scalex_best, scaley_best, a_best, b_best
            
    return params_best_upToNow

In [14]:
def optimal_params_slice_affine(init_cnt, scoremap, init_params=(0,0,0,1,1,0,0), 
                                iter_num=100, return_scores=False, lr=(.1,0,1e-6,0)):
    
    lr1, lr2, lr3, lr4 = lr
    
    score_best = 0
    
    xmin, ymin = np.min(init_cnt, axis=0)
    xmax, ymax = np.max(init_cnt, axis=0)
    cx, cy = np.mean(init_cnt, axis=0)
    
    xs, ys = np.meshgrid(np.arange(xmin, xmax+1), np.arange(ymin, ymax+1))
    pts = np.c_[xs.flat, ys.flat]
    nzs = pts[np.where(Path(init_cnt).contains_points(pts))[0]].T

    dSdxyz = np.gradient(scoremap, 3, 3)

    scores = []

    tx_best, ty_best, theta_best, sx_best, sy_best, a_best, b_best  = init_params

    for iteration in range(iter_num):

        dMdu = 0
        dMdv = 0
        dMdaz = 0
        dMdsx = 0
        dMdsy = 0
        dMda = 0
        dMdb = 0

        cos_z = np.cos(theta_best)
        sin_z = np.sin(theta_best)
        Rz_best = np.array([[sx_best*cos_z, -sx_best*sin_z], 
                            [sy_best*sin_z, sy_best*cos_z]])
        
        t = np.array([tx_best, ty_best])
        center = np.array([cx, cy])
        ds = nzs - center[:, np.newaxis]
                
        xs_prime, ys_prime = (np.dot([[1, a_best],[b_best, 1]], np.dot(Rz_best, ds)) + (t + center)[:,np.newaxis]).astype(np.int)

        valid = (xs_prime >= 0) & (ys_prime >= 0) & \
                (xs_prime < scoremap.shape[1]) & (ys_prime < scoremap.shape[0])

        if np.count_nonzero(valid) > 0:

            xs_prime_valid = xs_prime[valid]
            ys_prime_valid = ys_prime[valid]
            
            dxs, dys = ds
            dxs_valid = dxs[valid]
            dys_valid = dys[valid]

            xs, ys = nzs
            xs_valid = xs[valid]
            ys_valid = ys[valid]

            Sx = dSdxyz[1][ys_prime_valid, xs_prime_valid]
            Sy = dSdxyz[0][ys_prime_valid, xs_prime_valid]
            S = scoremap[ys_prime_valid, xs_prime_valid].sum()
            
#             fxx = sx_best*cos_z + a_best*sy_best*sin_z
#             fxy = -sx_best*sin_z + a_best*sy_best*cos_z
#             fyx = b_best*sx_best*cos_z + sy_best*sin_z
#             fyy = -b_best*sin_z*sx_best + sy_best*cos_z
            
#             dMdu += (np.dot(Sx, (-fxx+1)*dxs_valid) + np.dot(Sy, (-fyx+1)*dys_valid)) * sx_best * sy_best
#             dMdv += (np.dot(Sx, (-fxy+1)*dxs_valid) + np.dot(Sy, (-fyy+1)*dys_valid)) * sx_best * sy_best
            
            dMdu += Sx.sum() * sx_best * sy_best
            dMdv += Sy.sum() * sx_best * sy_best
            
            dx2dsx = dxs_valid*cos_z - dys_valid*sin_z
            dx2dsy = dxs_valid*a_best*sin_z + dys_valid*a_best*cos_z
            
            dy2dsx = dxs_valid*b_best*cos_z + dys_valid*(-b_best*sin_z)
            dy2dsy = dxs_valid*sin_z + dys_valid*cos_z
            
            dMdsx += ((np.dot(Sx, dx2dsx) + np.dot(Sy, dy2dsx)) * sx_best + .1*S) * sy_best
            dMdsy += ((np.dot(Sx, dx2dsy) + np.dot(Sy, dy2dsy)) * sy_best + .1*S) * sx_best
            

            dx2dtheta = dxs_valid*(-sx_best*sin_z+a_best*sy_best*cos_z) + \
                        dys_valid*(-sx_best*cos_z-a_best*sy_best*sin_z)
        
            dy2dtheta = dxs_valid*(-b_best*sin_z*sx_best+sy_best*cos_z) + \
                        dys_valid*(-b_best*sx_best*cos_z-sy_best*sin_z)

            dMdaz += (np.dot(Sx, dx2dtheta) + np.dot(Sy, dy2dtheta)) * sy_best * sx_best

#             dMdsx += np.dot(dSdxyz[1][ys_prime_valid, xs_prime_valid],
#                             (cos_z-b_best*sin_z)*xs_valid + (-a_best*cos_z-sin_z)*ys_valid) * scx_best * scy_best + \
#                     .5*scoremap[ys_prime_valid, xs_prime_valid].sum() * scy_best

#             dMdsy += np.dot(dSdxyz[0][ys_prime_valid, xs_prime_valid],
#                             (sin_z+b_best*cos_z)*xs_valid + (a_best*sin_z+cos_z)*ys_valid) * scx_best * scy_best + \
#                     .1*scoremap[ys_prime_valid, xs_prime_valid].sum() * scx_best
                
#             dMda += (np.dot(dSdxyz[1][ys_prime_valid, xs_prime_valid], -sin_z*sc_best*ys_valid) + \
#                     np.dot(dSdxyz[0][ys_prime_valid, xs_prime_valid], cos_z*sc_best*ys_valid)) * sc_best
                
#             dMdb += (np.dot(dSdxyz[1][ys_prime_valid, xs_prime_valid], -cos_z*sc_best*xs_valid) + \
#                     np.dot(dSdxyz[0][ys_prime_valid, xs_prime_valid], -sin_z*sc_best*xs_valid)) * sc_best

#         print '(dMdu, dMdv):', dMdu, dMdv
#         print '(dMdaz):', dMdaz
#         print '(dMdsx, dMdsy):', dMdsx, dMdsy
#         print '(dMda, dMdb):', dMda, dMdb

#         print 'lr1:', lr1, ', lr2:', lr2, ', lr3:', lr3, ', lr4:', lr4

#         lr1 = lr[0]*np.exp(-iteration/30.)
        
#         lr3 = lr[2]*np.exp(-iteration/30.)
        
        tx_best += lr1*dMdu
        ty_best += lr1*dMdv
        theta_best += lr2*dMdaz

        theta_best = theta_best%(2*np.pi)
        if theta_best > np.pi:
            theta_best -= 2*np.pi
        elif theta_best < -np.pi:
            theta_best += 2*np.pi
            
        sx_best += lr3*dMdsx
        sy_best += lr3*dMdsy
        
        sx_best = min(2., max(sx_best, 0.1))
        sy_best = min(2., max(sy_best, 0.1))
        
        a_best += lr4*dMda
        b_best += lr4*dMdb
    
#         print '(tx_best, ty_best):', tx_best, ty_best
#         print '(theta_best):', theta_best
#         print '(theta_best) in degrees:', np.rad2deg(theta_best)
#         print '(scx_best, scy_best):', sx_best, sy_best
#         print '(a, b):', a_best, b_best

        s = scoremap[ys_prime_valid, xs_prime_valid].sum() * sx_best * sy_best
#         s = scoremap[ys_prime_valid, xs_prime_valid].sum()

#         print 'score:', s
        scores.append(s)

#         print '\n'
        
        params_best_upToNow = (tx_best, ty_best, theta_best, sx_best, sy_best, a_best, b_best)

#         if s > score_best:
#             print 'Current best'
#             params_best_upToNow = (tx_best, ty_best, theta_best, sx_best, sy_best, a_best, b_best)
#             score_best = s
        
    best_finetune_params = params_best_upToNow
    
    if return_scores:
        return best_finetune_params, scores
    else:
        return best_finetune_params

In [15]:
labels_to_detect = ['5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

In [66]:
import cPickle as pickle
import datetime

In [74]:
autoAnnotations_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_autoAnnotations'

autoAnnotations_dir = os.path.join(autoAnnotations_rootdir, stack)
if not os.path.exists(autoAnnotations_dir):
    os.makedirs(autoAnnotations_dir)

In [80]:
# for sec in range(first_bs_sec, last_bs_sec+1):
# for sec in [158]:
for sec in range(150, 200):
# for sec in range(157, 167):

    print sec

    new_res = []

    for l in labels_to_detect:

        nbrs_containing_this_label = [(abs(nbr-sec), nbr) for i, nbr in enumerate(range(sec-3,sec)+range(sec+1,sec+4)) 
                if nbr in section_contains_label and l in section_contains_label[nbr]]

        if len(nbrs_containing_this_label) == 0:
            sys.stderr.write('No neighbor within three sections has annotation of class %s.\n' % l)
            continue

        closest_nbr = sorted(nbrs_containing_this_label)[0][1]

        print l

        dm = DataManager(stack=stack, section=closest_nbr)
        user, ts, _, res = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')

        lm = [lm for lm in res if lm['label'] == l][0]

        init_cnt = np.array(lm['vertices'])

        init_cnt = (init_cnt/8 - (399, 567)).astype(np.int)

        global scoremap
        scoremap = np.load('/oasis/projects/nsf/csd395/yuncong/CSHL_scoremaps_lossless/%(stack)s/%(sec)04d/%(stack)s_%(sec)04d_roi1_denseScoreMapLossless_%(label)s.npz' % \
                          {'stack': stack, 'sec': sec, 'label': l})['arr_0']

        scoremap = scoremap[::8, ::8][567:1316+1, 399:1778+1]


        params = grid_search_affine(init_cnt,iter_num=10, init_params=(0, 0, 0, 1., 1, 0, 0), 
                                   sigma_tx0=300, sigma_ty0=300, sigma_theta_z0=0, 
                                            sigma_scx0=0, sigma_scy0=0,
                                            sigma_a0=0, sigma_b0=0)

        best_finetune_params = optimal_params_slice_affine(init_cnt, scoremap, init_params=params,
                                                          iter_num=1000, return_scores=False, lr=(1e-2,1e-6,1e-6,0))


        tx_best, ty_best, theta_best, sx_best, sy_best, a_best, b_best = best_finetune_params


        xmin, ymin = np.min(init_cnt, axis=0)
        xmax, ymax = np.max(init_cnt, axis=0)
        cx, cy = np.mean(init_cnt, axis=0)
        center = np.array([cx, cy])

        cos_z = np.cos(theta_best)
        sin_z = np.sin(theta_best)
        R = np.array([[sx_best*cos_z, -sx_best*sin_z], 
                      [sy_best*sin_z, sy_best*cos_z]])

        t = np.array([tx_best, ty_best])

        xs, ys = (np.dot(np.dot([[1, a_best], [b_best, 1]], R), 
                         init_cnt.T - center[:, np.newaxis]) + (t+center)[:, np.newaxis]).astype(np.int)

        valid = (xs >= 0) & (ys >= 0) & (xs < scoremap.shape[1]) & (ys < scoremap.shape[0])

        new_cnt = np.c_[xs, ys][valid]


        new_lm = lm.copy()
        new_lm['vertices'] = (new_cnt + (399, 567)) * 8
        new_lm['labelPos'] = np.array(lm['labelPos']) + (tx_best*8, ty_best*8)

        new_lm['refVertices'] = lm['vertices']
        new_lm['refSection'] = closest_nbr

        new_res.append(new_lm)

    ##########################################

    
    timestamp = datetime.datetime.now().strftime("%m%d%Y%H%M%S")

    autoAnnotation_filepath = autoAnnotations_dir + '/%(stack)s_%(sec)04d_autoAnnotate_%(timestamp)s_consolidated.pkl' % \
                        {'stack': stack, 'sec': sec, 'timestamp': timestamp}

    pickle.dump(new_res, open(autoAnnotation_filepath, 'w'))
    
    
    ##########################################

    # viz1 = cropped_img
    # viz2 = plt.cm.hot(scoremap)
    # viz = img_as_ubyte(alpha_blending(viz1, viz2[...,:3], .9, 1.))


    dm = DataManager(stack=stack, section=sec)
    dm._load_image(versions=['rgb-jpg'])
    cropped_img = dm.image_rgb_jpg[::8, ::8][567:1316+1, 399:1778+1]

    viz = cropped_img.copy()

    for lm in new_res:

    # for pt in init_cnt:
    #     cv2.circle(viz, tuple(pt), 2, (0,255,0,255), -1)

        ref_cnt = np.array(lm['refVertices']).astype(np.int)/8-(399,567)
        for xy in ref_cnt:
            cv2.circle(viz, tuple(xy), 3, (0,255,0), -1)

        new_cnt = np.array(lm['vertices'])/8-(399,567)
        for xy in new_cnt:
            cv2.circle(viz, tuple(xy), 5, (255,0,0), -1)
        cv2.polylines(viz, [new_cnt.astype(np.int)], True, (255,0,0), 2)


        if sec in section_contains_label and lm['label'] in section_contains_label[sec]:
            dm = DataManager(stack=stack, section=sec)
            user, ts, _, res = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
            manual_cnt = [x['vertices'] for x in res if x['label'] == lm['label']][0]
            manual_cnt = np.array(manual_cnt)/8-(399,567)

            for xy in manual_cnt.astype(np.int):
                cv2.circle(viz, tuple(xy), 3, (0,0,255), -1)
            cv2.polylines(viz, [manual_cnt.astype(np.int)], True, (0,0,255), 2)


        lx, ly = lm['labelPos']/8 - (399, 567)
        cv2.putText(viz, lm['label'], (int(lx)-10, int(ly)+10), cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 3)

    cv2.imwrite(autoAnnotation_dir + '/%(stack)s_%(sec)04d_autoAnnotationViz.jpg' % \
                {'stack': stack, 'sec': sec}, viz[..., ::-1])

150
5N
VLL


No neighbor within three sections has annotation of class 7n.
No neighbor within three sections has annotation of class 7N.
No neighbor within three sections has annotation of class 12N.
No neighbor within three sections has annotation of class Gr.
No neighbor within three sections has annotation of class LVe.
No neighbor within three sections has annotation of class Pn.
No neighbor within three sections has annotation of class SuVe.


151
5N
7n
7N
VLL


No neighbor within three sections has annotation of class 12N.
No neighbor within three sections has annotation of class Gr.
No neighbor within three sections has annotation of class LVe.
No neighbor within three sections has annotation of class Pn.
No neighbor within three sections has annotation of class SuVe.


152
5N
7n
7N
VLL


...

No neighbor within three sections has annotation of class LVe.
No neighbor within three sections has annotation of class SuVe.
No neighbor within three sections has annotation of class VLL.
No neighbor within three sections has annotation of class 5N.


199
7n
7N
Gr


No neighbor within three sections has annotation of class 12N.


Pn


No neighbor within three sections has annotation of class LVe.
No neighbor within three sections has annotation of class SuVe.
No neighbor within three sections has annotation of class VLL.
